Unit 2 Assignment: Building a Mixture of Experts (MoE) Router

Name : Harsha Madev Hegde

SRN : PES2UG23CS212

Section D


Installing Dependencies

In [1]:
!pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.5 MB/s eta 0:00:00


Setting up API Key

In [2]:
# Secure API key input (hidden textbox)

from getpass import getpass
import os

# This creates a hidden input field
api_key = getpass("Enter GROQ API Key: ")

# Store in environment variable
os.environ["GROQ_API_KEY"] = api_key

print("✅ API key loaded securely.")

Enter GROQ API Key: ··········
✅ API key loaded securely.


Importing and Initializing Groq Client

In [3]:
from groq import Groq
import os

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

MODEL_NAME = "llama-3.1-8b-instant"

Defining Expert Configurations

In [4]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a Technical Support Expert. "
            "Be precise, logical, and code-focused. "
            "Explain bugs clearly and provide corrected code when necessary."
        ),
        "temperature": 0.7
    },
    "billing": {
        "system_prompt": (
            "You are a Billing Support Expert. "
            "Be empathetic, professional, and policy-driven. "
            "Explain billing issues and refund policies clearly."
        ),
        "temperature": 0.7
    },
    "general": {
        "system_prompt": (
            "You are a friendly general assistant. "
            "Respond helpfully and conversationally."
        ),
        "temperature": 0.7
    }
}

Router Function (Temperature = 0)

In [5]:
def route_prompt(user_input):
    routing_prompt = f"""
Classify the following user query into one of these categories:
[technical, billing, general]

Return ONLY the category name.

User Query:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are a strict classifier."},
            {"role": "user", "content": routing_prompt}
        ],
        temperature=0
    )

    category = response.choices[0].message.content.strip().lower()
    return category

Orchestrator Function

In [6]:
def process_request(user_input):
    category = route_prompt(user_input)

    # Safety fallback
    if category not in MODEL_CONFIG:
        category = "general"

    config = MODEL_CONFIG[category]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": config["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        temperature=config["temperature"]
    )

    return {
        "routed_to": category,
        "response": response.choices[0].message.content
    }

Testing With Example Inputs

In [7]:
# Example 1: Technical
print(process_request("My python script is throwing an IndexError on line 5."))

{'routed_to': 'technical', 'response': "To troubleshoot the issue, I'll need more information about your code. However, I can guide you through a general process.\n\n**General Steps to Troubleshoot an IndexError**\n\n1. **Identify the line number**: You've already mentioned that the error occurs on line 5. I'll make sure to refer to that line.\n2. **Check the indexing**: An IndexError typically occurs when you're trying to access an element in a list or other sequence that doesn't exist.\n3. **Verify the length of the sequence**: Make sure the sequence you're trying to access has at least as many elements as you're trying to access.\n\n**Example Code for a Common Scenario**\n\nHere's an example of a simple list with an IndexError:\n```python\nmy_list = [1, 2, 3]\nprint(my_list[3])  # This will raise an IndexError\n```\nIn this case, the error occurs because the list `my_list` only has three elements (at indices 0, 1, and 2), but we're trying to access the fourth element (index 3).\n\n*

In [8]:
# Example 2: Billing
print(process_request("I was charged twice for my subscription this month."))

{'routed_to': 'billing', 'response': "I'm so sorry to hear that you were charged twice for your subscription. That can be frustrating and confusing. \n\nI'd like to assure you that we take situations like this very seriously and I'm here to help resolve the issue for you. To better assist you, can you please provide me with some information? Could you tell me:\n\n1. The date you were initially charged for your subscription?\n2. The date you were charged the second time?\n3. The total amount charged on both occasions?\n4. The payment method used for the subscription?\n\nThis information will help me investigate the issue and ensure that the duplicate charge is removed from your account."}


In [9]:
# Example 3: General
print(process_request("Tell me something interesting about space."))

{'routed_to': 'general', 'response': "There's a fascinating fact about space that I'd love to share with you. Did you know that there's a giant storm on Jupiter that's been raging for at least 187 years? This massive storm is called the Great Red Spot, and it's a persistent anticyclonic storm that's so large it could swallow several Earths whole.\n\nThe Great Red Spot is a swirling storm of clouds that's about 10,000 miles wide and 25,000 miles long. It's a massive system of clouds that's been observed by astronomers for centuries, and it's still one of the most iconic features of the planet Jupiter.\n\nThe storm is thought to be a high-pressure region with strong winds that can reach speeds of up to 400 miles per hour. The exact cause of the storm's persistence is still a topic of debate among scientists, but it's believed to be due to a combination of Jupiter's internal heat and the planet's atmospheric circulation patterns.\n\nWhat's even more amazing is that the Great Red Spot is s

For user's input questions

In [ ]:
while True:
    user_input = input("\nAsk something (type 'exit' to quit): ")
    if user_input.lower() == "exit":
        break

    result = process_request(user_input)
    print("\nRouted To:", result["routed_to"])
    print("Response:", result["response"])


Ask something (type 'exit' to quit): tell me something about educatio

Routed To: general
Response: Education is a fascinating topic. Did you know that the concept of education dates back to ancient civilizations, with evidence of formal education systems in ancient Greece, Rome, and Egypt?

In ancient Greece, for example, education was reserved for the wealthy and focused on developing the mind through the study of philosophy, literature, and the arts. The famous philosopher Socrates is said to have believed that education should be a lifelong process of questioning and exploring ideas.

Fast-forward to the present day, and education has evolved significantly. We now have a much more inclusive and diverse education system, with access to education for people of all ages, backgrounds, and abilities.

Some interesting trends in modern education include:

1. **Online learning**: With the rise of the internet and digital technology, online learning has become increasingly popular. Many u

Final conclusion


In this assignment, we implemented a Mixture of Experts (MoE) architecture to build a Smart Customer Support Router

Assignment_MOE

.

The system uses an LLM-based router with temperature set to 0 to classify user queries into predefined categories (technical, billing, general). Based on the classification, the request is forwarded to a specialized expert configuration using different system prompts and a higher temperature for flexible responses.

This approach demonstrates how a single base LLM can simulate multiple domain experts through prompt engineering. The architecture is modular, scalable, and reflects real-world AI system design principles used in production environments.